In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import mlflow
import mlflow.pytorch

In [3]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///c:/Users/sanit/OneDrive/Escritorio/TP%20integrador%20NN/TP-Integrador-RN/mlruns/238164930765243842', creation_time=1749337243079, experiment_id='238164930765243842', last_update_time=1749337243079, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [4]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

KeyboardInterrupt: 

In [ ]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [ ]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [ ]:
# Crear directorio de logs
log_dir = "runs/mlp_experimento_1"
writer = SummaryWriter(log_dir=log_dir)


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [ ]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])


In [ ]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
# Paths
train_dir = r'skin-dataset-clasification/data/Split_smol/train/'
val_dir = r'skin-dataset-clasification/data/Split_smol/val/'

In [ ]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [ ]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

    writer.add_scalar("train/loss", train_loss, epoch)
    writer.add_scalar("train/accuracy", train_acc, epoch)

    # Log en MLflow
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

Epoch 1/10: 100%|██████████| 22/22 [00:07<00:00,  2.76it/s]
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being

Epoch 1:
  Train Loss: 2.7636, Accuracy: 28.59%
  Val   Loss: 1.7904, Accuracy: 41.11%


Epoch 2/10: 100%|██████████| 22/22 [00:08<00:00,  2.74it/s]


Epoch 2:
  Train Loss: 1.5558, Accuracy: 42.96%
  Val   Loss: 1.9246, Accuracy: 35.56%


Epoch 3/10: 100%|██████████| 22/22 [00:07<00:00,  2.81it/s]
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sanit\OneDrive\Escritorio\TP integrador NN\TP-Integrador-RN\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being

...

Epoch 8:
  Train Loss: 1.0329, Accuracy: 57.33%
  Val   Loss: 1.2422, Accuracy: 55.56%


Epoch 9/10: 100%|██████████| 22/22 [00:09<00:00,  2.25it/s]


Epoch 9:
  Train Loss: 0.9557, Accuracy: 64.94%
  Val   Loss: 1.3029, Accuracy: 53.33%


Epoch 10/10: 100%|██████████| 22/22 [00:09<00:00,  2.31it/s]


Epoch 10:
  Train Loss: 0.9376, Accuracy: 62.64%
  Val   Loss: 1.2358, Accuracy: 55.00%


In [ ]:
# Guardar modelo
torch.save(model.state_dict(), "mlp_model.pth")
print("Modelo guardado como 'mlp_model.pth'")
mlflow.log_artifact("mlp_model.pth")
mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
print("Modelo guardado como 'mlp_model.pth'")

Modelo guardado como 'mlp_model.pth'


2025/06/08 11:23:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model.pth'


In [ ]:
!tensorboard --logdir=runs/mlp_experimento_1

^C


In [ ]:
mlflow.end_run()

http://localhost:6006 para ver el tensorboard

## Acividades de Modificacion

### Dropout

In [ ]:
class MLPClassifierDropout(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)
    

In [ ]:
log_dir = "runs/mlp_experimento_2_dropout"
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
dropout_model=MLPClassifierDropout(num_classes=num_classes).to(device)
path=r'mlp_dropout_model.pth'
if os.path.exists(path):
    dropout_model.load_state_dict(torch.load(path))


Modelo cargado exitosaente


In [ ]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier with dropout",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
for epoch in range(n_epochs):
    dropout_model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

    writer.add_scalar("train/loss", train_loss, epoch)
    writer.add_scalar("train/accuracy", train_acc, epoch)

    # Log en MLflow
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

Epoch 1/10: 100%|██████████| 22/22 [00:09<00:00,  2.40it/s]


Epoch 1:
  Train Loss: 0.6608, Accuracy: 73.13%
  Val   Loss: 1.2747, Accuracy: 52.22%


Epoch 2/10: 100%|██████████| 22/22 [00:09<00:00,  2.42it/s]


Epoch 2:
  Train Loss: 0.6742, Accuracy: 73.56%
  Val   Loss: 1.3281, Accuracy: 52.22%


Epoch 3/10: 100%|██████████| 22/22 [00:08<00:00,  2.65it/s]


...

Epoch 8:
  Train Loss: 0.5811, Accuracy: 76.01%
  Val   Loss: 1.3374, Accuracy: 62.22%


Epoch 9/10: 100%|██████████| 22/22 [00:08<00:00,  2.55it/s]


Epoch 9:
  Train Loss: 0.6108, Accuracy: 77.16%
  Val   Loss: 1.3236, Accuracy: 55.56%


Epoch 10/10: 100%|██████████| 22/22 [00:08<00:00,  2.53it/s]


Epoch 10:
  Train Loss: 0.5539, Accuracy: 77.59%
  Val   Loss: 1.2478, Accuracy: 57.78%


In [ ]:
torch.save(dropout_model.state_dict(), "mlp_dropout_model.pth")
print("Modelo guardado como 'mlp_dropout_model.pth'")
mlflow.log_artifact("mlp_dropout_model.pth")
mlflow.pytorch.log_model(dropout_model, artifact_path="pytorch_model")
print("Modelo guardado como 'mlp_dropout_model.pth'")
mlflow.end_run()

Modelo guardado como 'mlp_dropout_model.pth'


2025/06/08 11:31:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_dropout_model.pth'


In [ ]:
!tensorboard --logdir=runs/mlp_experimento_2_dropout

^C


### Batch Normalization

In [ ]:
class MLPClassifierBatchNorm(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model =nn.Sequential(
         nn.Flatten(),
         nn.Linear(input_size, 512),
         nn.BatchNorm1d(512),
         nn.ReLU(),
         nn.Dropout(0.5),
         nn.Linear(512, 256),
         nn.BatchNorm1d(256),
         nn.ReLU(),
         nn.Dropout(0.5),
         nn.Linear(256, num_classes)
     )

    def forward(self, x):
        return self.model(x)

In [ ]:
log_dir = "runs/mlp_experimento_3_BatchNorm"
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
BatchNorm_model=MLPClassifierBatchNorm(num_classes=num_classes).to(device)
path=r'mlp_BatchNorm_model.pth'
if os.path.exists(path):
    dropout_model.load_state_dict(torch.load(path))


In [ ]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier with Batch normalization",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
for epoch in range(n_epochs):
    BatchNorm_model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

    writer.add_scalar("train/loss", train_loss, epoch)
    writer.add_scalar("train/accuracy", train_acc, epoch)

    # Log en MLflow
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

Epoch 1/10: 100%|██████████| 22/22 [00:09<00:00,  2.25it/s]


Epoch 1:
  Train Loss: 0.4744, Accuracy: 82.90%
  Val   Loss: 1.2843, Accuracy: 61.11%


Epoch 2/10: 100%|██████████| 22/22 [00:08<00:00,  2.60it/s]


Epoch 2:
  Train Loss: 0.4680, Accuracy: 82.61%
  Val   Loss: 1.3516, Accuracy: 58.89%


Epoch 3/10: 100%|██████████| 22/22 [00:09<00:00,  2.25it/s]


...

Epoch 8:
  Train Loss: 0.4262, Accuracy: 82.90%
  Val   Loss: 1.3362, Accuracy: 60.56%


Epoch 9/10: 100%|██████████| 22/22 [00:08<00:00,  2.52it/s]


Epoch 9:
  Train Loss: 0.4013, Accuracy: 84.20%
  Val   Loss: 1.5275, Accuracy: 54.44%


Epoch 10/10: 100%|██████████| 22/22 [00:08<00:00,  2.68it/s]


Epoch 10:
  Train Loss: 0.4743, Accuracy: 82.47%
  Val   Loss: 1.3840, Accuracy: 62.22%


In [ ]:
torch.save(BatchNorm_model.state_dict(), "mlp_BatchNorm_model.pth")
print("Modelo guardado como 'mlp_BatchNorm_model.pth'")
mlflow.log_artifact("mlp_BatchNorm_model.pth")
mlflow.pytorch.log_model(dropout_model, artifact_path="pytorch_model")
print("Modelo guardado como 'mlp_BatchNorm_model.pth'")
mlflow.end_run()

Modelo guardado como 'mlp_BatchNorm_model.pth'


KeyboardInterrupt: 

### Optimizador L2

In [ ]:
optimizer = torch.optim.Adam(BatchNorm_model.parameters(), lr=0.001, weight_decay=1e-4)

In [ ]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(p=0.3)
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)